<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/kapitza-pendulum-firebats2000/blob/main/kapitza_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Psuedocode

#1. Define dimensionlesss equation of motion:
* $\theta''+ 2\zeta\theta' + (\epsilon^2 - \alpha cos\tau)sin\theta = 0$
* $\tau = \omega_d t$

# 2. Define the ODE system (RHS function):
* Input: $\tau$, state vector $y = [\theta, \theta'], parameters (\epsilon, \alpha, \zeta)$
* Output: derivatives $[\theta',\theta'']$
* function kapitza_rhs($\tau, y, \epsilon, \alpha, \zeta$):
  * $\theta = y[0]$
  * $\theta' = y[1]$
  * $\theta'' = -2\zeta\theta' - (\epsilon^2 - \alpha cos(\tau))sin(\theta)$
* return [$\theta', \theta''$]

# 3.  Integrate using solve_ivp:
* sol = solve_ivp(kapitza_rhs, t_span = (0, tau_end), y0 = [theta0, theta_dot0], t_eval = tau_eval, args = (eps, alpha, zeta))
* Extarct $\theta(t)$: $\theta = sol.y[0]$
* Deiscard tansient portion:
  * index_start = index corresponding to N_periods_discard
  * theta_ss = theta[index_start:]
* compute stability measure

# 4.  Parameter sweep over $(\epsilon, \alpha)$
* eps_vals = array of $\epsilon$ values
* alpha_vals = array of $\alpha$ values
* create 2D array S to store diagnostics: S = zeros((len(alpha_vals), len(eps_vals)))

# 5.  Extract approximate stability boundry $\alpha_c(\epsilon)$
* alpha_c = array of lenght(eps_cals)
* For each epsilon index j:
  * column = S[:, j]
  * find first $\alpha_i$ where diagnostic < threshhold
  * if found: alpha_c[j] = alpha_vals[i]
  * else: alpha_c[j] = NaN

# 6.  Fit $\alpha_c = C\epsilon^2$
* valid_indices = where(alpha_c is finite)
* C= mean(alpha_c[valid]/(eps_vals[valid]^2))

# 7.  Plot 2D stability map
* Use matplotlib imshow or pcolormesh:
  * x-axis = $\epsilon$
  * y-axis = $\alpha$
  * color = diagnostic S[i, j]
* overlay fitted stability curve:
  * $\alpha_{fit}(\epsilon) = C\epsilon^2$
  * plot $\epsilon$ vs $\alpha_{fit}$

# 8. Print summary:
* Print fitted constant c
* Print $\alpha_c(\epsilon)$ table
* Print comments on stable vs unstable regions

In [ ]:
# Packages installation

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Define ODE (right hand side)
def kapitza_rhs(tau, y, eps, alpha, zeta):
    theta, theta_dot = y
    dtheta_dt = theta_dot
    dtheta_dot_dt = -2*zeta*theta_dot - (eps**2 - alpha*np.cos(tau))*np.sin(theta)
    return [dtheta_dt, dtheta_dot_dt]

In [ ]:
# Stability diagnostic

def compute_stability(eps, alpha, zeta,
                      theta0=np.pi + 1e-3,
                      theta_dot0=0,
                      N_periods_total=200,
                      N_periods_discard=150,
                      points_per_period=200):

    # integration time
    tau_end = N_periods_total * 2*np.pi
    tau_eval = np.linspace(0, tau_end, N_periods_total*points_per_period)

    sol = solve_ivp(
        kapitza_rhs,
        t_span=(0, tau_end),
        y0=[theta0, theta_dot0],
        t_eval=tau_eval,
        args=(eps, alpha, zeta),
        rtol=1e-7,
        atol=1e-9
    )

    thetas = sol.y[0]

    # discard transients
    discard_idx = int(len(tau_eval) * (N_periods_discard / N_periods_total))
    thetas_ss = thetas[discard_idx:]

    # diagnostic = std deviation of θ around π
    diag = np.std(thetas_ss - np.pi)

    return diag

In [3]:
# Parameter sweep

# Example sweep ranges
eps_vals  = np.linspace(0.5, 2.0, 10)
alpha_vals = np.linspace(0.0, 3.0, 25)

zeta = 0.05  # fixed damping

# Storage array
S = np.zeros((len(alpha_vals), len(eps_vals)))

print("Running parameter sweep...")
for i, alpha in enumerate(alpha_vals):
    for j, eps in enumerate(eps_vals):
        diag = compute_stability(eps, alpha, zeta)
        S[i, j] = diag

print("Sweep complete.")

Running parameter sweep...
Sweep complete.


In [4]:
# Exract stability boundary alpha_c(epsilon)

threshold = 0.1  # stability threshold for std(theta - π)

alpha_c = np.full(len(eps_vals), np.nan)

for j, eps in enumerate(eps_vals):
    col = S[:, j]
    stable_indices = np.where(col < threshold)[0]
    if len(stable_indices) > 0:
        alpha_c[j] = alpha_vals[stable_indices[0]]

# Fit α_c ≈ C ε²
mask = np.isfinite(alpha_c)
C_fit = np.mean(alpha_c[mask] / (eps_vals[mask]**2))

print(f"Fitted coefficient C ≈ {C_fit:.4f}")

Fitted coefficient C ≈ 0.0000
